<a href="https://colab.research.google.com/github/rhysdavies21/library/blob/master/rate_derivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ZC Bond Calculator

def zero_coupon_bond(par, y, t):
  '''
  Price of a zero coupon bond
  
  : param par: face value of bond
  : param y: annual yield or rate of the bond:
  : param t: time to maturity, in years
  '''
  return par / (1+y)**t

In [2]:
# Price ZC Bond

print(zero_coupon_bond(100, 0.05, 5))

78.35261664684589
